In [8]:
import requests
import json
import re
import string
from string import digits

In [14]:
def clean_str(text):
    text = re.sub('<.*?>'," ", text)
    text = text.lower()
    text = text.translate(string.maketrans("",""), string.punctuation)
    text = text.translate(None, digits)
    text = ' '.join( [w for w in text.split() if len(w)>1] )
    text = text.replace('§', '')
    text = text.replace( '  ', ' ' )
    text = text.replace( '    ', ' ' )
        
    return text.strip()

In [15]:
def get_data(page):
    r = requests.get("http://avoindata.eduskunta.fi/api/v1/tables/VaskiData/batch?pkStartValue=" + str(page) + "&pkName=Id")
    
    data = json.loads(r.text)
    
    return data['rowData']

In [16]:
def get_text(text_data):
    
    kappaleet = text_data.split('<sis:KappaleKooste>')
    text = ''
    
    for kpl in kappaleet:
        
        sisalto = kpl.split('</sis:KappaleKooste>')
        
        if len(sisalto) > 1:
            
            text += clean_str( sisalto[0].encode('utf-8') )
            text += ' '
    
    return text.strip()

In [17]:
def write_data(data, folder):
    
    for d in data:
        
        text = get_text(d[1])
        
        if text:
            with open('texts/' + folder + '/id_' + d[0] + '.txt', 'w') as f:
                f.write(text)

In [18]:
folder = 'clean'
start = 1
stop = 10000

for page in range(start, stop, 100):
    data = get_data(page)
    write_data(data, folder)